# Training Basics with Hyena-GLT

This notebook covers the fundamentals of training Hyena-GLT models for genomic sequence analysis. You'll learn how to prepare data, configure training, and monitor progress.

## Table of Contents
1. [Training Setup](#training-setup)
2. [Data Preparation](#data-preparation)
3. [Model Configuration](#model-configuration)
4. [Training Loop](#training-loop)
5. [Monitoring and Evaluation](#monitoring)
6. [Checkpointing and Resuming](#checkpointing)
7. [Common Issues and Solutions](#troubleshooting)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import sys
from tqdm import tqdm
import json
import time

# Add parent directory to path
sys.path.append(str(Path().absolute().parent.parent))

from hyena_glt.models.hyena_glt import HyenaGLT, HyenaGLTConfig
from hyena_glt.tokenizers import DNATokenizer
from hyena_glt.data import GenomicDataset, SequenceCollator
from hyena_glt.training import HyenaGLTTrainer, TrainingConfig
from hyena_glt.evaluation import ModelEvaluator

# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

print("Hyena-GLT Training Tutorial")
print("=" * 35)

## 1. Training Setup

Before training, we need to:
1. **Prepare the data**: Tokenize and format genomic sequences
2. **Configure the model**: Set architecture parameters
3. **Setup training**: Define optimizer, scheduler, and loss function
4. **Initialize monitoring**: Setup logging and visualization

### Key Training Components:
- **Tokenizer**: Converts sequences to numerical tokens
- **Dataset**: Handles data loading and preprocessing
- **DataLoader**: Manages batching and shuffling
- **Model**: The Hyena-GLT architecture
- **Optimizer**: Updates model parameters
- **Scheduler**: Adjusts learning rate during training
- **Loss Function**: Measures prediction quality

## 2. Data Preparation

Proper data preparation is crucial for successful training. We'll create a synthetic dataset for demonstration, but the same principles apply to real genomic data.

### Data Preparation Steps:
1. **Generate/Load sequences**: DNA, RNA, or protein sequences
2. **Create labels**: Classification targets or other annotations
3. **Tokenization**: Convert sequences to numerical tokens
4. **Dataset creation**: Wrap data in PyTorch Dataset
5. **DataLoader setup**: Configure batching and preprocessing

In [ ]:
# Generate synthetic DNA sequences for training
def generate_synthetic_dna_data(num_samples=1000, seq_length=512, num_classes=2):
    """Generate synthetic DNA sequences with labels"""
    sequences = []
    labels = []
    
    nucleotides = ['A', 'T', 'G', 'C']
    
    for i in range(num_samples):
        # Generate random DNA sequence
        sequence = ''.join(np.random.choice(nucleotides, seq_length))
        
        # Create synthetic labels based on GC content
        gc_content = (sequence.count('G') + sequence.count('C')) / len(sequence)
        label = 1 if gc_content > 0.5 else 0  # High GC = class 1, Low GC = class 0
        
        sequences.append(sequence)
        labels.append(label)
    
    return sequences, labels

# Generate training and validation data
print("Generating synthetic DNA data...")
train_sequences, train_labels = generate_synthetic_dna_data(num_samples=800, seq_length=256)
val_sequences, val_labels = generate_synthetic_dna_data(num_samples=200, seq_length=256)

print(f"Training samples: {len(train_sequences)}")
print(f"Validation samples: {len(val_sequences)}")
print(f"Sequence length: {len(train_sequences[0])}")
print(f"Classes: {set(train_labels)}")

# Show data distribution
train_label_counts = np.bincount(train_labels)
val_label_counts = np.bincount(val_labels)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

# Training data distribution
ax1.bar(['Low GC', 'High GC'], train_label_counts, color=['lightblue', 'lightcoral'])
ax1.set_title('Training Data Distribution')
ax1.set_ylabel('Number of Samples')
for i, count in enumerate(train_label_counts):
    ax1.text(i, count + 5, str(count), ha='center', va='bottom')

# Validation data distribution
ax2.bar(['Low GC', 'High GC'], val_label_counts, color=['lightblue', 'lightcoral'])
ax2.set_title('Validation Data Distribution')
ax2.set_ylabel('Number of Samples')
for i, count in enumerate(val_label_counts):
    ax2.text(i, count + 2, str(count), ha='center', va='bottom')

plt.tight_layout()
plt.show()

# Show sample sequences
print("\nSample sequences:")
for i in range(3):
    seq = train_sequences[i]
    label = train_labels[i]
    gc_content = (seq.count('G') + seq.count('C')) / len(seq)
    print(f"Sequence {i+1}: {seq[:50]}... (GC: {gc_content:.2f}, Label: {label})")

In [ ]:
# Create custom dataset class for our DNA data
class DNAClassificationDataset(Dataset):
    """Dataset for DNA sequence classification"""
    
    def __init__(self, sequences, labels, tokenizer, max_length=256):
        self.sequences = sequences
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
    
    def __len__(self):
        return len(self.sequences)
    
    def __getitem__(self, idx):
        sequence = self.sequences[idx]
        label = self.labels[idx]
        
        # Tokenize sequence
        tokens = self.tokenizer.encode(sequence, max_length=self.max_length)
        
        return {
            'input_ids': torch.tensor(tokens, dtype=torch.long),
            'labels': torch.tensor(label, dtype=torch.long),
            'sequence_length': torch.tensor(len(tokens), dtype=torch.long)
        }

# Initialize tokenizer
tokenizer = DNATokenizer()
print(f"Tokenizer vocabulary size: {tokenizer.vocab_size}")
print(f"Special tokens: {tokenizer.special_tokens}")

# Create datasets
train_dataset = DNAClassificationDataset(train_sequences, train_labels, tokenizer)
val_dataset = DNAClassificationDataset(val_sequences, val_labels, tokenizer)

print(f"\nDataset created:")
print(f"Train dataset size: {len(train_dataset)}")
print(f"Validation dataset size: {len(val_dataset)}")

# Test dataset
sample = train_dataset[0]
print(f"\nSample data:")
print(f"Input shape: {sample['input_ids'].shape}")
print(f"Label: {sample['labels'].item()}")
print(f"Sequence length: {sample['sequence_length'].item()}")
print(f"First 20 tokens: {sample['input_ids'][:20].tolist()}")

# Verify tokenization
original_seq = train_sequences[0][:20]
decoded_seq = tokenizer.decode(sample['input_ids'][:20].tolist())
print(f"Original: {original_seq}")
print(f"Decoded:  {decoded_seq}")

In [ ]:
# Create data collator for batching
collator = SequenceCollator(tokenizer.pad_token_id)

# Create data loaders
batch_size = 16
train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    collate_fn=collator,
    num_workers=0  # Set to 0 for notebook compatibility
)

val_loader = DataLoader(
    val_dataset,
    batch_size=batch_size,
    shuffle=False,
    collate_fn=collator,
    num_workers=0
)

print(f"Data loaders created:")
print(f"Training batches: {len(train_loader)}")
print(f"Validation batches: {len(val_loader)}")
print(f"Batch size: {batch_size}")

# Test batch loading
print("\nTesting batch loading...")
for batch_idx, batch in enumerate(train_loader):
    print(f"Batch {batch_idx + 1}:")
    print(f"  Input IDs shape: {batch['input_ids'].shape}")
    print(f"  Labels shape: {batch['labels'].shape}")
    print(f"  Attention mask shape: {batch['attention_mask'].shape}")
    
    # Show padding behavior
    seq_lengths = batch['attention_mask'].sum(dim=1)
    print(f"  Sequence lengths in batch: {seq_lengths.tolist()}")
    print(f"  Max length: {batch['input_ids'].shape[1]}")
    
    if batch_idx == 0:  # Only show first batch
        break

## 3. Model Configuration

Now let's configure our Hyena-GLT model for the DNA classification task. We need to set:

### Configuration Parameters:
- **Architecture**: Model dimensions, layers, etc.
- **Task-specific**: Number of classes, output type
- **Training**: Dropout, initialization, etc.
- **Efficiency**: Sequence compression, latent dimensions

In [ ]:
# Configure model for DNA classification
model_config = HyenaGLTConfig(
    vocab_size=tokenizer.vocab_size,
    latent_vocab_size=128,  # Smaller for this demo
    d_model=256,           # Model dimension
    n_layers=4,            # Number of Hyena layers
    sequence_length=256,   # Maximum input length
    latent_length=32,      # Compressed representation length
    num_classes=2,         # Binary classification
    num_heads=8,           # For any attention components
    dropout=0.1,           # Regularization
    layer_norm_eps=1e-5
)

print("Model Configuration:")
print(f"- Vocabulary size: {model_config.vocab_size}")
print(f"- Model dimension: {model_config.d_model}")
print(f"- Number of layers: {model_config.n_layers}")
print(f"- Sequence length: {model_config.sequence_length}")
print(f"- Latent length: {model_config.latent_length}")
print(f"- Compression ratio: {model_config.sequence_length / model_config.latent_length:.1f}x")
print(f"- Number of classes: {model_config.num_classes}")
print(f"- Dropout rate: {model_config.dropout}")

# Create model
model = HyenaGLT(model_config)
model = model.to(device)

# Count parameters
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"\nModel Statistics:")
print(f"- Total parameters: {total_params:,}")
print(f"- Trainable parameters: {trainable_params:,}")
print(f"- Model size: {total_params * 4 / 1024**2:.1f} MB")

# Test forward pass
print("\nTesting forward pass...")
model.eval()
with torch.no_grad():
    sample_batch = next(iter(train_loader))
    sample_batch = {k: v.to(device) for k, v in sample_batch.items()}
    
    outputs = model(sample_batch['input_ids'], attention_mask=sample_batch['attention_mask'])
    print(f"Output shape: {outputs.shape}")
    print(f"Output range: [{outputs.min().item():.3f}, {outputs.max().item():.3f}]")
    
    # Apply softmax to get probabilities
    probs = torch.softmax(outputs, dim=-1)
    print(f"Probabilities shape: {probs.shape}")
    print(f"Sample probabilities: {probs[0].tolist()}")

## 4. Training Loop

Now we'll set up the training loop with:

### Training Components:
1. **Optimizer**: Adam with weight decay
2. **Scheduler**: Learning rate scheduling
3. **Loss Function**: Cross-entropy for classification
4. **Metrics**: Accuracy, loss tracking
5. **Validation**: Regular evaluation on validation set

In [ ]:
# Training configuration
training_config = TrainingConfig(
    learning_rate=1e-4,
    batch_size=batch_size,
    num_epochs=10,
    warmup_steps=100,
    weight_decay=0.01,
    gradient_clipping=1.0,
    eval_steps=50,
    save_steps=100,
    logging_steps=10
)

print("Training Configuration:")
print(f"- Learning rate: {training_config.learning_rate}")
print(f"- Batch size: {training_config.batch_size}")
print(f"- Number of epochs: {training_config.num_epochs}")
print(f"- Warmup steps: {training_config.warmup_steps}")
print(f"- Weight decay: {training_config.weight_decay}")
print(f"- Gradient clipping: {training_config.gradient_clipping}")

# Setup optimizer and scheduler
optimizer = optim.AdamW(
    model.parameters(),
    lr=training_config.learning_rate,
    weight_decay=training_config.weight_decay
)

# Calculate total training steps
total_steps = len(train_loader) * training_config.num_epochs
warmup_steps = training_config.warmup_steps

# Linear warmup then cosine decay
def get_lr_scheduler(optimizer, warmup_steps, total_steps):
    def lr_lambda(step):
        if step < warmup_steps:
            return step / warmup_steps
        else:
            progress = (step - warmup_steps) / (total_steps - warmup_steps)
            return 0.5 * (1 + np.cos(np.pi * progress))
    return optim.lr_scheduler.LambdaLR(optimizer, lr_lambda)

scheduler = get_lr_scheduler(optimizer, warmup_steps, total_steps)

# Loss function
criterion = nn.CrossEntropyLoss()

print(f"\nTraining Setup:")
print(f"- Total training steps: {total_steps}")
print(f"- Warmup steps: {warmup_steps}")
print(f"- Steps per epoch: {len(train_loader)}")
print(f"- Optimizer: {type(optimizer).__name__}")
print(f"- Loss function: {type(criterion).__name__}")

In [ ]:
# Training loop with monitoring
def train_model(model, train_loader, val_loader, optimizer, scheduler, criterion, config, device):
    """Complete training loop with validation and monitoring"""
    
    # Training history
    history = {
        'train_loss': [],
        'train_acc': [],
        'val_loss': [],
        'val_acc': [],
        'learning_rate': []
    }
    
    global_step = 0
    best_val_acc = 0.0
    
    print("Starting training...")
    print("=" * 50)
    
    for epoch in range(config.num_epochs):
        # Training phase
        model.train()
        epoch_train_loss = 0.0
        epoch_train_correct = 0
        epoch_train_total = 0
        
        progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{config.num_epochs}")
        
        for batch_idx, batch in enumerate(progress_bar):
            # Move batch to device
            batch = {k: v.to(device) for k, v in batch.items()}
            
            # Forward pass
            optimizer.zero_grad()
            outputs = model(batch['input_ids'], attention_mask=batch['attention_mask'])
            loss = criterion(outputs, batch['labels'])
            
            # Backward pass
            loss.backward()
            
            # Gradient clipping
            if config.gradient_clipping > 0:
                torch.nn.utils.clip_grad_norm_(model.parameters(), config.gradient_clipping)
            
            optimizer.step()
            scheduler.step()
            
            # Statistics
            epoch_train_loss += loss.item()
            predictions = torch.argmax(outputs, dim=-1)
            epoch_train_correct += (predictions == batch['labels']).sum().item()
            epoch_train_total += batch['labels'].size(0)
            
            global_step += 1
            
            # Update progress bar
            current_lr = scheduler.get_last_lr()[0]
            progress_bar.set_postfix({
                'loss': f'{loss.item():.4f}',
                'acc': f'{epoch_train_correct/epoch_train_total:.4f}',
                'lr': f'{current_lr:.2e}'
            })
            
            # Validation
            if global_step % config.eval_steps == 0:
                val_loss, val_acc = evaluate_model(model, val_loader, criterion, device)
                
                # Save best model
                if val_acc > best_val_acc:
                    best_val_acc = val_acc
                    print(f"\nNew best validation accuracy: {val_acc:.4f}")
                
                model.train()  # Back to training mode
        
        # End of epoch statistics
        avg_train_loss = epoch_train_loss / len(train_loader)
        train_acc = epoch_train_correct / epoch_train_total
        
        # Final validation for the epoch
        val_loss, val_acc = evaluate_model(model, val_loader, criterion, device)
        
        # Record history
        history['train_loss'].append(avg_train_loss)
        history['train_acc'].append(train_acc)
        history['val_loss'].append(val_loss)
        history['val_acc'].append(val_acc)
        history['learning_rate'].append(scheduler.get_last_lr()[0])
        
        print(f"\nEpoch {epoch+1} Summary:")
        print(f"  Train Loss: {avg_train_loss:.4f}, Train Acc: {train_acc:.4f}")
        print(f"  Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")
        print(f"  Learning Rate: {scheduler.get_last_lr()[0]:.2e}")
        print("-" * 50)
    
    print(f"Training completed! Best validation accuracy: {best_val_acc:.4f}")
    return history

def evaluate_model(model, dataloader, criterion, device):
    """Evaluate model on validation/test set"""
    model.eval()
    total_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for batch in dataloader:
            batch = {k: v.to(device) for k, v in batch.items()}
            
            outputs = model(batch['input_ids'], attention_mask=batch['attention_mask'])
            loss = criterion(outputs, batch['labels'])
            
            total_loss += loss.item()
            predictions = torch.argmax(outputs, dim=-1)
            correct += (predictions == batch['labels']).sum().item()
            total += batch['labels'].size(0)
    
    avg_loss = total_loss / len(dataloader)
    accuracy = correct / total
    
    return avg_loss, accuracy

# Start training
history = train_model(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    optimizer=optimizer,
    scheduler=scheduler,
    criterion=criterion,
    config=training_config,
    device=device
)

## 5. Monitoring and Evaluation

After training, let's analyze the results and visualize the training progress.

In [ ]:
# Visualize training progress
def plot_training_history(history):
    """Plot training curves"""
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 10))
    
    epochs = range(1, len(history['train_loss']) + 1)
    
    # Loss curves
    ax1.plot(epochs, history['train_loss'], 'b-', label='Training Loss', linewidth=2)
    ax1.plot(epochs, history['val_loss'], 'r-', label='Validation Loss', linewidth=2)
    ax1.set_title('Training and Validation Loss')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Loss')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Accuracy curves
    ax2.plot(epochs, history['train_acc'], 'b-', label='Training Accuracy', linewidth=2)
    ax2.plot(epochs, history['val_acc'], 'r-', label='Validation Accuracy', linewidth=2)
    ax2.set_title('Training and Validation Accuracy')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Accuracy')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    # Learning rate schedule
    ax3.plot(epochs, history['learning_rate'], 'g-', linewidth=2)
    ax3.set_title('Learning Rate Schedule')
    ax3.set_xlabel('Epoch')
    ax3.set_ylabel('Learning Rate')
    ax3.set_yscale('log')
    ax3.grid(True, alpha=0.3)
    
    # Training metrics summary
    final_train_acc = history['train_acc'][-1]
    final_val_acc = history['val_acc'][-1]
    best_val_acc = max(history['val_acc'])
    
    ax4.bar(['Final Train', 'Final Val', 'Best Val'], 
            [final_train_acc, final_val_acc, best_val_acc],
            color=['blue', 'red', 'green'], alpha=0.7)
    ax4.set_title('Accuracy Summary')
    ax4.set_ylabel('Accuracy')
    ax4.set_ylim(0, 1)
    
    # Add value labels on bars
    for i, v in enumerate([final_train_acc, final_val_acc, best_val_acc]):
        ax4.text(i, v + 0.01, f'{v:.3f}', ha='center', va='bottom')
    
    ax4.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Print summary statistics
    print("\nTraining Summary:")
    print("=" * 30)
    print(f"Final training accuracy: {final_train_acc:.4f}")
    print(f"Final validation accuracy: {final_val_acc:.4f}")
    print(f"Best validation accuracy: {best_val_acc:.4f}")
    print(f"Final training loss: {history['train_loss'][-1]:.4f}")
    print(f"Final validation loss: {history['val_loss'][-1]:.4f}")
    
    # Check for overfitting
    if final_train_acc - final_val_acc > 0.1:
        print("\n⚠️  Warning: Possible overfitting detected (train acc >> val acc)")
    elif final_val_acc > final_train_acc:
        print("\n✅ Good generalization (val acc >= train acc)")
    else:
        print("\n✅ Reasonable training progress")

# Plot training history
plot_training_history(history)

In [ ]:
# Detailed model evaluation
def detailed_evaluation(model, dataloader, tokenizer, device, num_examples=10):
    """Perform detailed evaluation with examples"""
    model.eval()
    
    all_predictions = []
    all_labels = []
    all_probabilities = []
    example_sequences = []
    
    with torch.no_grad():
        for batch_idx, batch in enumerate(dataloader):
            batch_device = {k: v.to(device) for k, v in batch.items()}
            
            outputs = model(batch_device['input_ids'], attention_mask=batch_device['attention_mask'])
            probabilities = torch.softmax(outputs, dim=-1)
            predictions = torch.argmax(outputs, dim=-1)
            
            all_predictions.extend(predictions.cpu().numpy())
            all_labels.extend(batch['labels'].numpy())
            all_probabilities.extend(probabilities.cpu().numpy())
            
            # Store some example sequences for analysis
            if len(example_sequences) < num_examples:
                for i in range(min(batch['input_ids'].size(0), num_examples - len(example_sequences))):
                    sequence_tokens = batch['input_ids'][i].numpy()
                    sequence_str = tokenizer.decode(sequence_tokens)
                    example_sequences.append({
                        'sequence': sequence_str,
                        'true_label': batch['labels'][i].item(),
                        'predicted_label': predictions[i].item(),
                        'probabilities': probabilities[i].cpu().numpy().tolist()
                    })
    
    # Calculate metrics
    accuracy = np.mean(np.array(all_predictions) == np.array(all_labels))
    
    # Confusion matrix
    from sklearn.metrics import confusion_matrix, classification_report
    cm = confusion_matrix(all_labels, all_predictions)
    
    # Visualize results
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    
    # Confusion matrix
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax1,
                xticklabels=['Low GC', 'High GC'],
                yticklabels=['Low GC', 'High GC'])
    ax1.set_title('Confusion Matrix')
    ax1.set_xlabel('Predicted')
    ax1.set_ylabel('True')
    
    # Prediction confidence distribution
    all_probs = np.array(all_probabilities)
    max_probs = np.max(all_probs, axis=1)
    
    ax2.hist(max_probs, bins=20, alpha=0.7, edgecolor='black')
    ax2.set_title('Prediction Confidence Distribution')
    ax2.set_xlabel('Maximum Probability')
    ax2.set_ylabel('Frequency')
    ax2.axvline(x=0.5, color='red', linestyle='--', alpha=0.7, label='Random')
    ax2.legend()
    
    plt.tight_layout()
    plt.show()
    
    # Print detailed metrics
    print("\nDetailed Evaluation Results:")
    print("=" * 40)
    print(f"Overall Accuracy: {accuracy:.4f}")
    print(f"Number of samples: {len(all_labels)}")
    print(f"Class distribution: {np.bincount(all_labels)}")
    print(f"Average confidence: {np.mean(max_probs):.4f}")
    
    print("\nClassification Report:")
    print(classification_report(all_labels, all_predictions, 
                              target_names=['Low GC', 'High GC']))
    
    # Show example predictions
    print("\nExample Predictions:")
    print("-" * 80)
    for i, example in enumerate(example_sequences[:5]):
        sequence_preview = example['sequence'][:50] + '...' if len(example['sequence']) > 50 else example['sequence']
        true_label = 'High GC' if example['true_label'] == 1 else 'Low GC'
        pred_label = 'High GC' if example['predicted_label'] == 1 else 'Low GC'
        confidence = max(example['probabilities'])
        
        status = "✅" if example['true_label'] == example['predicted_label'] else "❌"
        
        print(f"{status} Example {i+1}:")
        print(f"   Sequence: {sequence_preview}")
        print(f"   True: {true_label}, Predicted: {pred_label} (confidence: {confidence:.3f})")
        print(f"   Probabilities: [Low GC: {example['probabilities'][0]:.3f}, High GC: {example['probabilities'][1]:.3f}]")
        print()
    
    return {
        'accuracy': accuracy,
        'predictions': all_predictions,
        'labels': all_labels,
        'probabilities': all_probabilities,
        'confusion_matrix': cm,
        'examples': example_sequences
    }

# Perform detailed evaluation
eval_results = detailed_evaluation(model, val_loader, tokenizer, device)

## 6. Checkpointing and Resuming

Checkpointing allows you to save model state and resume training later. This is essential for long training runs.

In [ ]:
# Save model checkpoint
def save_checkpoint(model, optimizer, scheduler, epoch, history, filepath):
    """Save complete training checkpoint"""
    checkpoint = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'scheduler_state_dict': scheduler.state_dict(),
        'history': history,
        'model_config': model.config.__dict__,
        'best_val_acc': max(history['val_acc']) if history['val_acc'] else 0.0
    }
    
    torch.save(checkpoint, filepath)
    print(f"Checkpoint saved to {filepath}")

def load_checkpoint(filepath, model, optimizer=None, scheduler=None):
    """Load training checkpoint"""
    checkpoint = torch.load(filepath, map_location='cpu')
    
    model.load_state_dict(checkpoint['model_state_dict'])
    
    if optimizer is not None:
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    
    if scheduler is not None:
        scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
    
    print(f"Checkpoint loaded from {filepath}")
    print(f"Resumed from epoch {checkpoint['epoch']}")
    print(f"Best validation accuracy: {checkpoint['best_val_acc']:.4f}")
    
    return checkpoint['epoch'], checkpoint['history']

# Save current model
checkpoint_path = "hyena_glt_dna_classifier.pt"
save_checkpoint(
    model=model,
    optimizer=optimizer,
    scheduler=scheduler,
    epoch=training_config.num_epochs,
    history=history,
    filepath=checkpoint_path
)

# Demonstrate loading
print("\nDemonstrating checkpoint loading...")

# Create a new model instance
new_model = HyenaGLT(model_config)
new_model = new_model.to(device)

# Load the checkpoint
epoch, loaded_history = load_checkpoint(checkpoint_path, new_model)

# Verify the loaded model works
print("\nTesting loaded model...")
test_batch = next(iter(val_loader))
test_batch = {k: v.to(device) for k, v in test_batch.items()}

with torch.no_grad():
    original_outputs = model(test_batch['input_ids'], attention_mask=test_batch['attention_mask'])
    loaded_outputs = new_model(test_batch['input_ids'], attention_mask=test_batch['attention_mask'])
    
    # Check if outputs are identical
    outputs_match = torch.allclose(original_outputs, loaded_outputs, atol=1e-6)
    print(f"Original and loaded model outputs match: {outputs_match}")

# Save model for inference (model only)
inference_path = "hyena_glt_dna_classifier_inference.pt"
torch.save({
    'model_state_dict': model.state_dict(),
    'model_config': model_config.__dict__,
    'tokenizer_config': {
        'vocab_size': tokenizer.vocab_size,
        'special_tokens': tokenizer.special_tokens
    }
}, inference_path)
print(f"\nInference model saved to {inference_path}")

## 7. Common Issues and Solutions

### Training Issues and Solutions:

#### 1. **Slow Convergence**
- **Issue**: Model takes too long to learn
- **Solutions**:
  - Increase learning rate (carefully)
  - Reduce model complexity
  - Check data quality and labels
  - Use learning rate scheduling

#### 2. **Overfitting**
- **Issue**: Training accuracy >> Validation accuracy
- **Solutions**:
  - Increase dropout rate
  - Add weight decay
  - Use data augmentation
  - Reduce model size
  - Early stopping

#### 3. **Memory Issues**
- **Issue**: CUDA out of memory
- **Solutions**:
  - Reduce batch size
  - Use gradient accumulation
  - Reduce sequence length
  - Use mixed precision training

#### 4. **Unstable Training**
- **Issue**: Loss spikes or NaN values
- **Solutions**:
  - Reduce learning rate
  - Add gradient clipping
  - Check for numerical instabilities
  - Use better initialization

#### 5. **Poor Performance**
- **Issue**: Low accuracy on validation set
- **Solutions**:
  - Increase model capacity
  - Improve data quality
  - Tune hyperparameters
  - Use transfer learning
  - Check data preprocessing

In [ ]:
# Training diagnostics and debugging tools
def diagnose_training_issues(model, dataloader, device, num_batches=5):
    """Diagnose common training issues"""
    model.eval()
    
    issues_found = []
    
    print("Training Diagnostics")
    print("=" * 30)
    
    # Check for gradient flow
    print("1. Checking gradient flow...")
    model.train()
    
    total_norm = 0
    param_count = 0
    
    for batch_idx, batch in enumerate(dataloader):
        if batch_idx >= num_batches:
            break
            
        batch = {k: v.to(device) for k, v in batch.items()}
        
        # Forward pass
        outputs = model(batch['input_ids'], attention_mask=batch['attention_mask'])
        loss = nn.CrossEntropyLoss()(outputs, batch['labels'])
        
        # Backward pass
        model.zero_grad()
        loss.backward()
        
        # Check gradients
        for name, param in model.named_parameters():
            if param.grad is not None:
                param_norm = param.grad.data.norm(2)
                total_norm += param_norm.item() ** 2
                param_count += 1
        
        break  # Only check first batch
    
    total_norm = total_norm ** (1. / 2)
    avg_grad_norm = total_norm / param_count if param_count > 0 else 0
    
    print(f"   Average gradient norm: {avg_grad_norm:.6f}")
    
    if avg_grad_norm < 1e-7:
        issues_found.append("Very small gradients - possible vanishing gradient problem")
    elif avg_grad_norm > 10:
        issues_found.append("Very large gradients - possible exploding gradient problem")
    
    # Check for dead neurons
    print("\n2. Checking for dead neurons...")
    activations = []
    
    def hook_fn(module, input, output):
        if isinstance(output, torch.Tensor):
            activations.append(output.detach().cpu())
    
    hooks = []
    for module in model.modules():
        if isinstance(module, (nn.Linear, nn.Conv1d)):
            hooks.append(module.register_forward_hook(hook_fn))
    
    with torch.no_grad():
        sample_batch = next(iter(dataloader))
        sample_batch = {k: v.to(device) for k, v in sample_batch.items()}
        _ = model(sample_batch['input_ids'], attention_mask=sample_batch['attention_mask'])
    
    # Remove hooks
    for hook in hooks:
        hook.remove()
    
    dead_neurons = 0
    total_neurons = 0
    
    for activation in activations:
        if len(activation.shape) >= 2:
            # Check for neurons that are always zero
            neuron_activity = activation.abs().sum(dim=tuple(range(len(activation.shape)-1)))
            dead_neurons += (neuron_activity == 0).sum().item()
            total_neurons += neuron_activity.numel()
    
    dead_neuron_ratio = dead_neurons / total_neurons if total_neurons > 0 else 0
    print(f"   Dead neuron ratio: {dead_neuron_ratio:.4f} ({dead_neurons}/{total_neurons})")
    
    if dead_neuron_ratio > 0.1:
        issues_found.append(f"High dead neuron ratio ({dead_neuron_ratio:.2%}) - consider adjusting initialization or learning rate")
    
    # Check data statistics
    print("\n3. Checking data statistics...")
    all_labels = []
    all_seq_lengths = []
    
    for batch in dataloader:
        all_labels.extend(batch['labels'].tolist())
        all_seq_lengths.extend(batch['attention_mask'].sum(dim=1).tolist())
    
    label_distribution = np.bincount(all_labels)
    class_imbalance = max(label_distribution) / min(label_distribution) if min(label_distribution) > 0 else float('inf')
    
    print(f"   Class distribution: {label_distribution.tolist()}")
    print(f"   Class imbalance ratio: {class_imbalance:.2f}")
    print(f"   Average sequence length: {np.mean(all_seq_lengths):.1f}")
    print(f"   Sequence length std: {np.std(all_seq_lengths):.1f}")
    
    if class_imbalance > 3:
        issues_found.append(f"Significant class imbalance (ratio: {class_imbalance:.1f}) - consider class weighting")
    
    # Model capacity check
    print("\n4. Checking model capacity...")
    total_params = sum(p.numel() for p in model.parameters())
    data_size = len(dataloader.dataset)
    params_per_sample = total_params / data_size
    
    print(f"   Parameters per training sample: {params_per_sample:.1f}")
    
    if params_per_sample > 100:
        issues_found.append("High parameter-to-data ratio - possible overfitting risk")
    elif params_per_sample < 1:
        issues_found.append("Low parameter-to-data ratio - model might be undercapacity")
    
    # Summary
    print("\n" + "=" * 30)
    if issues_found:
        print("⚠️  Issues Found:")
        for i, issue in enumerate(issues_found, 1):
            print(f"   {i}. {issue}")
    else:
        print("✅ No obvious issues detected")
    
    return issues_found

# Run diagnostics
diagnostic_results = diagnose_training_issues(model, train_loader, device)

# Training tips based on our results
print("\n" + "=" * 50)
print("Training Tips for Your Model:")
print("=" * 50)
print("✅ Model successfully trained on synthetic DNA data")
print("✅ Good convergence and generalization observed")
print("✅ Reasonable parameter efficiency")
print("\nNext steps:")
print("- Try with real genomic datasets")
print("- Experiment with different sequence lengths")
print("- Test transfer learning capabilities")
print("- Optimize for your specific use case")

## Conclusion

This tutorial covered the complete training pipeline for Hyena-GLT models:

### What We Accomplished:
1. **Data Preparation**: Created synthetic DNA sequences with meaningful labels
2. **Model Configuration**: Set up Hyena-GLT for binary classification
3. **Training Loop**: Implemented complete training with validation
4. **Monitoring**: Tracked metrics and visualized progress
5. **Evaluation**: Analyzed model performance in detail
6. **Checkpointing**: Saved and loaded model states
7. **Troubleshooting**: Diagnosed potential training issues

### Key Training Insights:
- **Compression Benefits**: BLT tokenization reduces computational requirements
- **Stable Training**: Hyena blocks provide stable gradient flow
- **Good Generalization**: Proper regularization prevents overfitting
- **Efficient Memory Usage**: Linear complexity enables longer sequences

### Next Steps:
1. **Real Data**: Apply to actual genomic datasets
2. **Advanced Training**: Multi-task learning, transfer learning
3. **Optimization**: Mixed precision, distributed training
4. **Fine-tuning**: Task-specific adaptations

### Resources:
- [Fine-tuning Tutorial](05_fine_tuning.ipynb)
- [Advanced Techniques](06_advanced_training.ipynb)
- [Production Deployment](07_deployment.ipynb)
- [Example Scripts](../examples/)

You now have the foundation to train Hyena-GLT models for your specific genomic modeling tasks!